In [1]:
import requests
from bs4 import BeautifulSoup
import os
import cloudscraper

import shutil
import time
import re
from tqdm import tqdm

ModuleNotFoundError: No module named 'requests'

In [ ]:
URL = r"https://www.ralphlauren.nl/en/men/clothing/hoodies-sweatshirts/10204?webcat=men%7Cclothing%7Cmen-clothing-hoodies-sweatshirts"